#  Juliaで高速なベクトル・行列の区間演算をしたい

ベクトルの内積、行列ベクトル積、行列積の区間演算はとても重要。すなわち
$x, y\in\mathbb{IF}^n$, $A, B\in\mathbb{IF}^{n\times n}$ に対して、それぞれ$x^Ty \subset \alpha$, $Ax\subset z$, $AB\subset C$となる$\alpha\in\mathbb{IF}$, $z\in\mathbb{IF}^n$, $C\in\mathbb{IF}^{n\times n}$を求める方法を考える。

## 素朴な方法

最も素朴な方法は各演算を区間演算に変更した方式である。

$$
    \alpha = \sum_{k = 1}^n x_ky_k
$$

$$
    z_i = \sum_{k = 1}^n A_{i,k}x_k\quad (1\le i\le n)
$$

$$
    C_{ij} = \sum_{k+1}^n A_{i,k}B_{k,j}\quad (1\le i,j\le n)
$$

特にJuliaでは、計算を関数で包む事で、高速な計算ができることがあるらしい。


In [2]:
using IntervalArithmetic, BenchmarkTools

n = 1000;
x = randn(n);
y = randn(n);

x_int = map(Interval, x);
y_int = map(Interval, y);

@time alpha = dot(x_int,y_int)

# setrounding(Interval, :fast)

# function dot_vec()
#     n = 1000; x = randn(n); y = randn(n);
#     x_int = map(Interval, x); y_int = map(Interval, y);
#     alpha = dot(x_int,y_int);
# end
# @btime dot_vec()

  0.138869 seconds (447.67 k allocations: 18.914 MiB, 7.94% gc time)


In [3]:
using LinearAlgebra, Test
n = 1000; A = randn(n,n); x = randn(n); 
A_int = map(Interval, A); x_int = map(Interval, x); y = similar(x_int);
@time z = A_int*x_int;
@time mul!(y,A_int,x_int)
println(y ⊆ z)
# @test z ⊆ y

# @btime z = A_int*x_int;

  0.672312 seconds (2.30 M allocations: 85.068 MiB, 4.30% gc time)
  0.041477 seconds (20 allocations: 1.562 KiB)
true


In [5]:
n = 100;
A = randn(n,n);
B = randn(n,n);
A_int = map(Interval, A);
B_int = map(Interval, B);
@time C = A_int*B_int;
@time C = A_int*B_int;
@time C = A_int*B_int;

  0.050053 seconds (8 allocations: 156.656 KiB)
  0.046273 seconds (8 allocations: 156.656 KiB)
  0.046111 seconds (8 allocations: 156.656 KiB)


In [ ]:
# using StaticArrays
# # A = randn(n, n);
# # B = randn(n, n);
# A_int = SMatrix{n,n}(map(Interval, A));
# B_int = SMatrix{n,n}(map(Interval, B));
# @time A_int*B_int;
# @time A_int*B_int;
# @time A_int*B_int;

In [41]:
n = 1000; A = randn(n,n); B = randn(n,n);
A_int = map(Interval, A); B_int = map(Interval, B);
@btime C = A_int*B_int;

  41.813 s (8 allocations: 15.26 MiB)


これを高速化したい。考えられる方法は

- 並列化したい（[Loopvectrization](https://github.com/chriselrod/LoopVectorization.jl), [Tullio](https://github.com/mcabbott/Tullio.jl)?, @simd命令）
- [StaticArrays](https://github.com/JuliaArrays/StaticArrays.jl_)を使うと[速いらしい](https://qiita.com/cometscome_phys/items/669273a49ab2417d3af8)
- BLASを使う、だけど丸め方向が変えられないので、丸めの向きを変えない区間演算
    - 内積は森倉定理8あるいは教科書の定理2.4(p.38)
    - 行列ベクトル積は？
    - 行列積は森倉4節
 
 
 ## BLASを使う
 
 まずJuliaの中で、BLASを使うには``LinearAlgebra``モジュールが必要。

In [3]:
using LinearAlgebra, BenchmarkTools
# versioninfo()
# BLAS.vendor()
n = 5000;
A, B, C = randn(n,n), randn(n,n), zeros(n,n);

@btime C = A*B;
@btime mul!($C, $A, $B);

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
  826.653 ms (2 allocations: 190.73 MiB)
  839.978 ms (0 allocations: 0 bytes)


**定義**

$\mathbf{u}=2^{-53}$を倍精度の単位相対丸めとする。$\mathbf{S}_{\min}=2^{-1074}$を倍精度浮動小数点数の正の最小数とする。$\mathbf{F}_{\min}=2^{-1022}$を倍精度浮動小数点数の正規化数正の最小数とする。 

In [7]:
println(bitstring(1.0))
println(bitstring(1.0-2^-53))
println(bitstring(2^-1074))
println(bitstring(2^-1022))
u = 2.0^-53;
eta = 2.0^-1074;
realmin = 2.0^-1022;

0011111111110000000000000000000000000000000000000000000000000000
0011111111101111111111111111111111111111111111111111111111111111
0000000000000000000000000000000000000000000000000000000000000001
0000000000010000000000000000000000000000000000000000000000000000


In [2]:
function [C_mid,C_rad]=mm_ufp(A_mid,A_rad,B_mid,B_rad)
%現在の丸めモードは最近点であると仮定

    %system_dependent('setround',0.5)
    u=2^-53;
    n=length(A_mid);

    if(2*(n+2)*u>=1)
        error('mm_ufp failed!(2(n+2)u>=1)'),
    end
    
    C_mid = A_mid*B_mid;
    C_rad = (n+2)*u*ufp(abs(A_mid)*abs(B_mid))+realmin;
    
    AmBr=abs(A_mid)*B_rad;
    AmBr=succ_near(AmBr+((n+2)*u*ufp(AmBr)+realmin));
    
    ArBm=A_rad*abs(B_mid);
    ArBm=succ_near(ArBm+((n+2)*u*ufp(ArBm)+realmin));
    
    ArBr=A_rad*B_rad;
    ArBr=succ_near(ArBr+((n+2)*u*ufp(ArBr)+realmin));
    
    rad_sum=C_rad+AmBr+ArBm+ArBr;
    clear AmBr ArBm ArBr
    
    C_rad = succ_near((rad_sum)+3*u*ufp(rad_sum));

    clear rad_sum
    
end

function S = ufp(P)
    
    u=2^-53;
    a=2^52+1;
    Q=a*P;
    T=(1-u)*Q;
    S=abs(Q-T);

end

function [succ] = succ_near(c)

eta=2^-1074;
u=2^-53;
phi=u*(1+2*u);
if abs(c)>=(1/2)*u^-2*eta
e=phi*abs(c);
succ=c+e;
elseif abs(c)<u^-1*eta
succ=c+eta;
else
C=u^-1*c;
e=phi*abs(C);
succ=(C+e)*u;
end
end




:mkl